# Prerequisites

- ecdsa
- LLL
- **hidden number problem**
    - https://kel.bz/post/hnp/

# Theory

- https://www.youtube.com/watch?v=6ssTlSSIJQE&t
- https://eprint.iacr.org/2019/023.pdf

## Setup

Let
- $m$ be a message with its hash $h$
- $q$ be the order of the generator point $G$
- $P = d*G$ where $d$ is the secret key
- $(s, r)$ be a signature with  $r = x(k*G)$ and $s \equiv k^{-1}(h + rd) \ mod \ q$

Conside multiple message-signature pairs => we can make the system of equations

$\begin{align}
k_1 - s_1^{-1}r_1d - s_1^{-1}h_1 \equiv 0 \ mod \ q \\ 
k_2 - s_2^{-1}r_2d - s_2^{-1}h_1 \equiv 0 \ mod \ q \\ 
\vdots \\
k_m - s_m^{-1}r_md - s_m^{-1}h_1 \equiv 0 \ mod \ q \\ 
\end{align}$

with unknows $k_1, ... k_m, d$

**Task**
- If $k_i$ are small enough we can attack this system with lattice techniques to find $d$

*Intuition*:
There is likely that the system has 1 solution and the LLL can find it


## The lattice basis

- https://static.aminer.org/pdf/PDF/000/119/803/hardness_of_computing_the_most_significant_bits_of_secret_keys.pdf

Let 

$\begin{align}
k_1 - t_1d - a_1 \equiv 0 \ mod \ q \\ 
k_2 - t_2d - a_2 \equiv 0 \ mod \ q \\ 
\vdots \\
k_m - t_md - a_m \equiv 0 \ mod \ q \\ 
\end{align}$

be a system of equations with unknows $k_1, ... k_m, d$

Let the matrix be:

$M = \begin{bmatrix}
q & \\
& q &\\
&  & \ddots \\
&  & & q  \\
t_1 & t_2 & \dots & t_m
\end{bmatrix}
$

Solve CVP with $v_t = (a_1, ... a_m)$. Then $v_k = (k_1, ..., k_m)$ will be the distance

*Intuition*: Since the nonces are small => $v_k$ will be small (the shortest distance)

We like SVP better than CVP => Let's make a new lattice basis:
$M = \begin{bmatrix}
q & \\
& q &\\
&  & \ddots \\
&  & & q  \\
t_1 & t_2 & \dots & t_m & B / q \\
a_1 & a_2 & \dots & a_m & &B 
\end{bmatrix}
$

where
- $|k_i| < B$; $B$ = some bound
- $v_b = (k_1, ... k_m, Bd/ q, B)$ is a short vector that we search for

**Remark**:
- The scaling factors are not that easy to pick sometimes

How does this work?
- $\dim L = m+2$
- $\det L = B^2q^{m-1}$
- Ignoring approx LLL will find $|v| \leq (\det L )^{1/\dim L}$
- We are searching for a vector with length $v_k \leq B\sqrt{m + 2}$ => we expect to find $v_k$ when $\log B \leq \lfloor \log q(m-1)/m - (\log m) / 2 \rfloor$

More signatures => The bigger nonce $k$ can be to find him

| msg-sign pair | k bits | 
| ---| --- | 
| 2 | 128 |
| 3 | 170 |
| 4 | 190 |
| 20 | 242 |
| 40 | 248 |

if $k_1$ and $k_2$ have the same MSB but different from 0 we can make a third signature with $k_1 -k_2 = 0$

# Code

In [1]:
from hashlib import sha1, sha256
from Crypto.Util.number import bytes_to_long, long_to_bytes, inverse
from ecdsa import ellipticcurve
from ecdsa.ecdsa import curve_256, generator_256, Public_key, Private_key
from random import randint

In [119]:
G = generator_256
q = int(G.order())
p = int(curve_256.p())

In [398]:
def short_biased_k(r_list, s_list, h_list, q, p, B):
    #construct the system of equations
    t_list = [inverse(s, q) * r % q for (s,r) in zip(s_list, r_list)]
    a_list = [inverse(s, q) * h % q for (s,h) in zip(s_list, h_list)]
    
    #construct the lattice
    m = len(a_list)
    M = matrix(QQ, m+2, m+2)
    for ii in range(m):
        M[ii, ii] = q
        M[-2, ii] = t_list[ii]
        M[-1, ii] = a_list[ii]
    M[-2, -2] = QQ(B) / QQ(q)
    M[-1, -1] = QQ(B)
    
    #LLL
    M_lll = M.LLL(delta = .75)
    
    #get the shortest vector
    for v in M_lll:
        if v[-1] == B:
            v_short = v
            break
    k0 = v_short[0]
    
    #find d
    d = inverse(r_list[0], q) * (k0*s_list[0] - h_list[0]) % q 
    
    #other way to find d
    for row in M_lll:
        d2 = ((QQ(row[-2]) * q) / B) % q
        print(row.norm().n())
        print('d2', d2)
        if d2 == d:
            break
            
    for row in M_lll:
        print(row.norm().n())
            
    return d
    
    

In [188]:
d = randint(1,q-1)
pubkey = Public_key(G, d*G)
privkey = Private_key(pubkey, d)
print(d)

12084444493995898434781940200479723954920185454074318905945108975382097768244


In [189]:
m_list = [b'secret_message', b'really_secret', b'impossible_to_decrypt']
r_list = []
s_list = []
h_list = []
k_list = [] #we'll check later
for msg in m_list:
    h = bytes_to_long(sha256(msg).digest())
    h_list.append(h)
    k = randint(1, 2^160)
    k_list.append(k)
    sig = privkey.sign(h, k)
    r_list.append(int(sig.r))
    s_list.append(int(sig.s))
m = len(m_list)

In [190]:
print((log(q, 2)*(m - 1) / m - log(m, 2) / 2).n()) #we need a b with less than 170b

169.874185416082


In [440]:
B = randint(1, 2^174)
log(B, 2).n()

173.111165961449

In [441]:
k_list, B

([90307492798281668742503182550835776780328118172,
  65066592534692662974177996459644158574477076978,
  30350456303370736524481350735706608108194432435],
 12931638000416933232535240450888344577493279416607167)

In [442]:
d

12084444493995898434781940200479723954920185454074318905945108975382097768244

In [443]:
d_decr = short_biased_k(r_list, s_list, h_list, q, p,B)

2.66129722597643e51
d2 21016977260407140673891506766005218885848673264276298447185318002418591747200
3.74251472046688e51
d2 114138050609754041016482145282172767530859322748215969783977314626140005313097
4.07030511532204e51
d2 100031548778080393491225322764107865380132388892239809366091526693783200739331
4.54571322254317e51
d2 15963639736451830795659295700086989836737695835134615577350254684108833750531
1.30018710293960e52
d2 12084444493995898434781940200479723954920185454074318905945108975382097768244
2.66129722597643e51
3.74251472046688e51
4.07030511532204e51
4.54571322254317e51
1.30018710293960e52


In [444]:
d_decr

12084444493995898434781940200479723954920185454074318905945108975382097768244

In [447]:
B * sqrt(m + 2).n()

2.89160216293517e52

In [445]:
d == d_decr

True

# Resources

- https://www.youtube.com/watch?v=6ssTlSSIJQE&t
- https://crypto.stackexchange.com/questions/44644/how-does-the-biased-k-attack-on-ecdsa-work
- https://www.youtube.com/watch?v=Prq6AHJz6SY&t